In [1]:
!pip install textarena smithery anthropic httpx mcp --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: textarena
    Found existing installation: TextArena 0.5.6
    Uninstalling TextArena-0.5.6:
      Successfully uninstalled TextArena-0.5.6

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [1]:
import os
os.environ["ANTHROPIC_API_KEY"] = "sk..."
os.environ["E2B_API_KEY"] = "e2b..."

In [2]:
from textarena.core import Agent
import textarena as ta
import asyncio
from typing import Optional

STANDARD_GAME_PROMPT = "You are a competitive game player. Make sure you read the game instructions carefully, and always follow the required format."

class AsyncAnthropicAgent(Agent):
    """Agent class using the Anthropic Claude API to generate responses asynchronously."""
    def __init__(self, model_name: str, system_prompt: Optional[str] = STANDARD_GAME_PROMPT, max_tokens: int = 1000, temperature: float = 0.9, verbose: bool = False):
        """
        Initialize the Anthropic agent.

        Args:
            model_name (str): The name of the Claude model (e.g., "claude-3-5-sonnet-20241022").
            system_prompt (Optional[str]): The system prompt to use (default: STANDARD_GAME_PROMPT).
            max_tokens (int): The maximum number of tokens to generate.
            temperature (float): The temperature for randomness in response generation.
            verbose (bool): If True, additional debug info will be printed.
        """
        super().__init__()
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.verbose = verbose
        
        try:
            import anthropic
        except ImportError:
            raise ImportError(
                "Anthropic package is required for AsyncAnthropicAgent. "
                "Install it with: pip install anthropic"
            )
            
        self.client = anthropic.AsyncAnthropic()
    
    async def _make_request(self, observation: str) -> str:
        """Make a single API request to Anthropic and return the generated message."""
        response = await self.client.messages.create(
            model=self.model_name,
            max_tokens=self.max_tokens,
            temperature=self.temperature,
            system=self.system_prompt,
            messages=[
                {"role": "user", "content": [{"type": "text", "text": observation}]}
            ]
        )
        
        return response.content[0].text.strip()
    
    async def _retry_request(self, observation: str, retries: int = 3, delay: int = 5) -> str:
        """
        Attempt to make an API request with retries.

        Args:
            observation (str): The input to process.
            retries (int): The number of attempts to try.
            delay (int): Seconds to wait between attempts.

        Raises:
            Exception: The last exception caught if all retries fail.
        """
        last_exception = None
        for attempt in range(1, retries + 1):
            try:
                response = await self._make_request(observation)
                if self.verbose:
                    print(f"\nObservation: {observation}\nResponse: {response}")
                return response
            except Exception as e:
                last_exception = e
                print(f"Attempt {attempt} failed with error: {e}")
                if attempt < retries:
                    await asyncio.sleep(delay)
        raise last_exception
    
    async def __call__(self, observation: str) -> str:
        """
        Process the observation using the Anthropic API and return the generated response.
        
        Args:
            observation (str): The input string to process.
        
        Returns:
            str: The generated response.
        """
        if not isinstance(observation, str):
            raise ValueError(f"Observation must be a string. Received type: {type(observation)}")
        return await self._retry_request(observation)

In [9]:
import textarena as ta
import smithery
import mcp
import os
import json


async def create_session(url: str):
    async with smithery.websocket_client(url) as streams:
        async with mcp.client.session.ClientSession(*streams) as session:
            yield session

class MCPAgent(AsyncAnthropicAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.url = smithery.create_smithery_url(
            "wss://server.smithery.ai/e2b/ws", {"e2bApiKey": os.environ["E2B_API_KEY"]}
        )

        self.session = None
        self._session_generator = None

    async def _initialize_session(self):
        if self._session_generator is None:
            self._session_generator = create_session(self.url)
        
        if self.session is None:
            self.session = await anext(self._session_generator)
        
        return self.session

    async def _make_request(self, observation: str) -> str:
        """Make a single API request to Anthropic and return the generated message."""
        session = await self._initialize_session()

        try:
            tools_result = await session.list_tools()
            tools = tools_result.model_dump()["tools"]

            tools = [
                {"input_schema": tool.pop("inputSchema"), **tool}
                for tool in tools
                if "inputSchema" in tool
            ]

            print("Available tools:", tools)

            final_response_text = ""
            is_tool_call_pending = True
            messages = [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": observation}],
                }
            ]

            # Loop to handle multiple tool calls in a conversation
            while is_tool_call_pending:
                response = await self.client.messages.create(
                    model=self.model_name,
                    max_tokens=self.max_tokens,
                    temperature=self.temperature,
                    system=self.system_prompt,
                    messages=messages,
                    tools=tools,
                )

                print("Response:", response)

                # Check if there's a tool_use in the response
                is_tool_call_pending = False
                for content_block in response.content:
                    if content_block.type == "tool_use":
                        is_tool_call_pending = True

                        tool_name = content_block.name
                        tool_input = content_block.input
                        tool_id = content_block.id

                        print(f"Tool called: {tool_name}")
                        print(f"Tool input: {json.dumps(tool_input, indent=2)}")

                        # Execute the tool using MCP session
                        try:
                            tool_result = await session.call_tool(
                                tool_name, tool_input
                            )

                            # Convert tool result to string format for Anthropic
                            # The content must be a string, not an object
                            tool_result_dict = tool_result.model_dump()
                        except Exception as e:
                            if "MCP error" in str(e):
                                tool_result_dict = {"error": str(e)}

                        result_str = json.dumps(tool_result_dict)
                        print(f"Tool result: {result_str}")

                        # Add tool call and result to messages
                        messages.append(
                            {
                                "role": "assistant",
                                "content": [content_block.model_dump()],
                            }
                        )

                        # Add tool response to messages - content must be a string
                        messages.append(
                            {
                                "role": "user",
                                "content": [
                                    {
                                        "type": "tool_result",
                                        "tool_use_id": tool_id,
                                        "content": result_str,  # Now it's a string
                                    }
                                ],
                            }
                        )
                    elif content_block.type == "text":
                        # Accumulate text responses
                        final_response_text += content_block.text

                # If no tool calls were made, we use the text response
                if not is_tool_call_pending and not final_response_text:
                    final_response_text = response.content[0].text

        except Exception as e:

            print(f"Error: {e}")
            raise e

        return final_response_text.strip()

In [ ]:
import textarena as ta

# Initialize agents
agents = {
    0: MCPAgent(model_name="claude-3-7-sonnet-20250219"),
    1: AsyncAnthropicAgent(model_name="claude-3-5-haiku-20241022"),
}

# Initialize environment from subset and wrap it
env = ta.make(env_id="SimpleNegotiation-v0")
env = ta.wrappers.LLMObservationWrapper(env=env)
env = ta.wrappers.SimpleRenderWrapper(
    env=env,
    player_names={0: "sonnet", 1: "haiku"},
)

env.reset(num_players=len(agents))
done = False
while not done:
    player_id, observation = env.get_observation()
    # Alternative if running outside notebook:
    # action = asyncio.get_event_loop().run_until_complete(agents[player_id](observation))
    action = await agents[player_id](observation)
    done, info = env.step(action=action)
    print("step complete")
rewards = env.close()
print(rewards)

Available tools: [{'input_schema': {'type': 'object', 'properties': {'code': {'type': 'string'}}, 'required': ['code'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, 'name': 'run_code', 'description': 'Run python code in a secure sandbox by E2B. Using the Jupyter Notebook syntax.'}]
Response: Message(id='msg_014SnBW11PSd38q7gmKmuiR2', content=[TextBlock(citations=None, text="I'll help you navigate this Negotiation Game. Let me first analyze your resources and their values to understand what would be most beneficial to trade.", type='text'), ToolUseBlock(id='toolu_01L6erW5oRhUHNDAQeDq4hbt', input={'code': '# Current resources and their values\nresources = {\n    "Wheat": {"qty": 10, "value": 6},\n    "Wood": {"qty": 25, "value": 10},\n    "Sheep": {"qty": 19, "value": 18},\n    "Brick": {"qty": 5, "value": 29},\n    "Ore": {"qty": 20, "value": 35}\n}\n\n# Calculate total value of current resources\ntotal_value = sum(res["qty"] * res["value"] for r

╭────────────────────────────────────────────────── Game State ───────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─────────────── Basic Information ───────────────╮ ╭─────────────────── player_resources ────────────────────╮ │
│ │                                                 │ │                                                         │ │
│ │                ╷                                │ │          ╷       ╷     ╷       ╷       ╷                │ │
│ │   Key          │ Value                          │ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood           │ │
│ │  ══════════════╪═══════                         │ │  ════════╪═══════╪═════╪═══════╪═══════╪══════          │ │
│ │   current_turn │ 1                              │ │   sonnet │ 5     │ 20  │ 19    │ 10    │ 25             │ │
│ │   max_turns    │ 10                             │ │   haiku  │ 15    │ 12  │ 14    │ 9     │ 8              │ │
│ │                ╵                                │ │          ╵       ╵     ╵       ╵       ╵                │ │
│ │                                                 │ │                                                         │ │
│ ╰─────────────────────────────────────────────────╯ ╰─────────────────────────────────────────────────────────╯ │
│ ╭──────────────── inventory_value ────────────────╮                                                             │
│ │                                                 │                                                             │
│ │          ╷        ╷         ╷                   │                                                             │
│ │   Player │ change │ current │ initial           │                                                             │
│ │  ════════╪════════╪═════════╪═════════          │                                                             │
│ │   sonnet │ 0      │ 1497    │ 1497              │                                                             │
│ │   haiku  │ 0      │ 1044    │ 1044              │                                                             │
│ │          ╵        ╵         ╵                   │                                                             │
│ │                                                 │                                                             │
│ ╰─────────────────────────────────────────────────╯                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────── Game State ────────────────────────╮╭──────────────────── Game Log ─────────────────────╮
│                                                            ││                                                   │
│ ╭────────────────── Basic Information ───────────────────╮ ││ [GAME]: Game started.                             │
│ │                                                        │ ││ sonnet: I'll help you navigate this Negotiation   │
│ │                ╷                                       │ ││ Game. Let me first analyze your resources and     │
│ │   Key          │ Value                                 │ ││ their values to                                   │
│ │  ══════════════╪═══════                                │ ││ understand what would be most beneficial to       │
│ │   current_turn │ 1                                     │ ││ trade.I'll analyze my position in the Negotiation │
│ │   max_turns    │ 10                                    │ ││ Game:                                             │
│ │                ╵                                       │ ││                                                   │
│ │                                                        │ ││ Hello! I'm looking to make some mutually          │
│ ╰────────────────────────────────────────────────────────╯ ││ beneficial trades. Looking at my resources, I     │
│ ╭─────────────────── player_resources ───────────────────╮ ││ have plenty of Wood, Ore,                         │
│ │                                                        │ ││ and Sheep, but I'm a bit short on Brick which has │
│ │          ╷       ╷     ╷       ╷       ╷               │ ││ high value.                                       │
│ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood          │ ││                                                   │
│ │  ════════╪═══════╪═════╪═══════╪═══════╪══════         │ ││ My strategy will be to trade my lower-value       │
│ │   sonnet │ 5     │ 20  │ 19    │ 10    │ 25            │ ││ resources (like Wheat and Wood) to get more       │
│ │   haiku  │ 15    │ 12  │ 14    │ 9     │ 8             │ ││ high-value resources (like                        │
│ │          ╵       ╵     ╵       ╵       ╵               │ ││ Brick and Ore). This should maximize my total     │
│ │                                                        │ ││ value.                                            │
│ ╰────────────────────────────────────────────────────────╯ ││                                                   │
│ ╭─────────────────── inventory_value ────────────────────╮ ││ [Offer: 3 Wood, 2 Wheat -> 1 Brick]               │
│ │                                                        │ ││                                                   │
│ │          ╷        ╷         ╷                          │ ││ This offer would be beneficial for me as I'm      │
│ │   Player │ change │ current │ initial                  │ ││ trading 3 Wood (value 30) and 2 Wheat (value 12)  │
│ │  ════════╪════════╪═════════╪═════════                 │ ││ for 1 Brick (value                                │
│ │   sonnet │ 0      │ 1497    │ 1497                     │ ││ 29). It's close to equal value, but I'm           │
│ │   haiku  │ 0      │ 1044    │ 1044                     │ ││ consolidating my resources into higher-value      │
│ │          ╵        ╵         ╵                          │ ││ items, which is my goal.                          │
│ │                                                        │ ││                                                   │
│ ╰────────────────────────────────────────────────────────╯ ││ Let me know if you're interested in this trade or │
│                                                            ││ if you have a counter-offer!                      │
│                                                       

step complete


╭────────────────────────────────────────────────── Game State ───────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─────────────── Basic Information ───────────────╮ ╭─────────────────── player_resources ────────────────────╮ │
│ │                                                 │ │                                                         │ │
│ │                ╷                                │ │          ╷       ╷     ╷       ╷       ╷                │ │
│ │   Key          │ Value                          │ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood           │ │
│ │  ══════════════╪═══════                         │ │  ════════╪═══════╪═════╪═══════╪═══════╪══════          │ │
│ │   current_turn │ 2                              │ │   sonnet │ 5     │ 20  │ 19    │ 10    │ 25             │ │
│ │   max_turns    │ 10                             │ │   haiku  │ 15    │ 12  │ 14    │ 9     │ 8              │ │
│ │                ╵                                │ │          ╵       ╵     ╵       ╵       ╵                │ │
│ │                                                 │ │                                                         │ │
│ ╰─────────────────────────────────────────────────╯ ╰─────────────────────────────────────────────────────────╯ │
│ ╭──────────────── inventory_value ────────────────╮                                                             │
│ │                                                 │                                                             │
│ │          ╷        ╷         ╷                   │                                                             │
│ │   Player │ change │ current │ initial           │                                                             │
│ │  ════════╪════════╪═════════╪═════════          │                                                             │
│ │   sonnet │ 0      │ 1497    │ 1497              │                                                             │
│ │   haiku  │ 0      │ 1044    │ 1044              │                                                             │
│ │          ╵        ╵         ╵                   │                                                             │
│ │                                                 │                                                             │
│ ╰─────────────────────────────────────────────────╯                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────── Game State ────────────────────────╮╭──────────────────── Game Log ─────────────────────╮
│                                                            ││                                                   │
│ ╭────────────────── Basic Information ───────────────────╮ ││ + [Wheat]   Qty: 11   Value: 6    Total: 66       │
│ │                                                        │ ││ + [Wood]    Qty: 25   Value: 10   Total: 250      │
│ │                ╷                                       │ ││ + [Sheep]   Qty: 19   Value: 18   Total: 342      │
│ │   Key          │ Value                                 │ ││ + [Brick]   Qty: 8    Value: 29   Total: 232      │
│ │  ══════════════╪═══════                                │ ││ + [Ore]     Qty: 18   Value: 35   Total: 630      │
│ │   current_turn │ 7                                     │ ││                                                   │
│ │   max_turns    │ 10                                    │ ││ Your resources are clearly ranked by value per    │
│ │                ╵                                       │ ││ unit:                                             │
│ │                                                        │ ││ 1. Ore (35) - highest value                       │
│ ╰────────────────────────────────────────────────────────╯ ││ 2. Brick (29) - high value                        │
│ ╭─────────────────── player_resources ───────────────────╮ ││ 3. Sheep (18) - medium value                      │
│ │                                                        │ ││ 4. Wood (10) - low value                          │
│ │          ╷       ╷     ╷       ╷       ╷               │ ││ 5. Wheat (6) - lowest value                       │
│ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood          │ ││                                                   │
│ │  ════════╪═══════╪═════╪═══════╪═══════╪══════         │ ││ Based on this analysis, I would recommend:        │
│ │   sonnet │ 10    │ 18  │ 19    │ 8     │ 20            │ ││                                                   │
│ │   haiku  │ 10    │ 14  │ 14    │ 11    │ 13            │ ││ [Offer: 5 Wood, 3 Wheat -> 2 Brick]               │
│ │          ╵       ╵     ╵       ╵       ╵               │ ││                                                   │
│ │                                                        │ ││ This trade would be beneficial because:           │
│ ╰────────────────────────────────────────────────────────╯ ││ - You're offering 5 Wood (value 50) + 3 Wheat     │
│ ╭─────────────────── inventory_value ────────────────────╮ ││ (value 18) = total 68                             │
│ │                                                        │ ││ - You're requesting 2 Brick (value 58)            │
│ │          ╷        ╷         ╷                          │ ││ - While the pure value exchange slightly favors   │
│ │   Player │ change │ current │ initial                  │ ││ Player 1, you're consolidating lower-value        │
│ │  ════════╪════════╪═════════╪═════════                 │ ││ resources into                                    │
│ │   sonnet │ 13     │ 1510    │ 1497                     │ ││ higher-value ones                                 │
│ │   haiku  │ 16     │ 1060    │ 1044                     │ ││ - You have plenty of Wood to spare (25 units)     │
│ │          ╵        ╵         ╵                          │ ││ - You're increasing your Brick holdings, which is │
│ │                                                        │ ││ your second most valuable resource                │
│ ╰────────────────────────────────────────────────────────╯ ││                                                   │
│                                                            ││ This aligns with strategy #1 and #2: trading      │
│                                                       

step complete


╭────────────────────────────────────────────────── Game State ───────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─────────────── Basic Information ───────────────╮ ╭─────────────────── player_resources ────────────────────╮ │
│ │                                                 │ │                                                         │ │
│ │                ╷                                │ │          ╷       ╷     ╷       ╷       ╷                │ │
│ │   Key          │ Value                          │ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood           │ │
│ │  ══════════════╪═══════                         │ │  ════════╪═══════╪═════╪═══════╪═══════╪══════          │ │
│ │   current_turn │ 8                              │ │   sonnet │ 10    │ 20  │ 17    │ 8     │ 16             │ │
│ │   max_turns    │ 10                             │ │   haiku  │ 10    │ 12  │ 16    │ 11    │ 17             │ │
│ │                ╵                                │ │          ╵       ╵     ╵       ╵       ╵                │ │
│ │                                                 │ │                                                         │ │
│ ╰─────────────────────────────────────────────────╯ ╰─────────────────────────────────────────────────────────╯ │
│ ╭──────────────── inventory_value ────────────────╮                                                             │
│ │                                                 │                                                             │
│ │          ╷        ╷         ╷                   │                                                             │
│ │   Player │ change │ current │ initial           │                                                             │
│ │  ════════╪════════╪═════════╪═════════          │                                                             │
│ │   sonnet │ 7      │ 1504    │ 1497              │                                                             │
│ │   haiku  │ 20     │ 1064    │ 1044              │                                                             │
│ │          ╵        ╵         ╵                   │                                                             │
│ │                                                 │                                                             │
│ ╰─────────────────────────────────────────────────╯                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────── Game State ────────────────────────╮╭──────────────────── Game Log ─────────────────────╮
│                                                            ││                                                   │
│ ╭────────────────── Basic Information ───────────────────╮ ││ Strategic Analysis:                               │
│ │                                                        │ ││ 1. Resource Value Comparison:                     │
│ │                ╷                                       │ ││    - Offered resources: 5 Wood (50) + 3 Wheat     │
│ │   Key          │ Value                                 │ ││ (18) = 68                                         │
│ │  ══════════════╪═══════                                │ ││    - Requested Brick (2 units): 2 * 29 = 58       │
│ │   current_turn │ 8                                     │ ││    - Slight value difference in my favor          │
│ │   max_turns    │ 10                                    │ ││                                                   │
│ │                ╵                                       │ ││ 2. Current Resource Position:                     │
│ │                                                        │ ││    - I have 25 Wood (surplus)                     │
│ ╰────────────────────────────────────────────────────────╯ ││    - I have 11 Wheat (surplus)                    │
│ ╭─────────────────── player_resources ───────────────────╮ ││    - I have 8 Brick after previous trade          │
│ │                                                        │ ││    - The trade helps consolidate lower-value      │
│ │          ╷       ╷     ╷       ╷       ╷               │ ││ resources                                         │
│ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood          │ ││                                                   │
│ │  ════════╪═══════╪═════╪═══════╪═══════╪══════         │ ││ 3. Trade Considerations:                          │
│ │   sonnet │ 10    │ 20  │ 17    │ 8     │ 16            │ ││    - The trade is close to value-neutral          │
│ │   haiku  │ 10    │ 12  │ 16    │ 11    │ 17            │ ││    - I would be converting surplus low-value      │
│ │          ╵       ╵     ╵       ╵       ╵               │ ││ resources (Wood, Wheat) into a higher-value       │
│ │                                                        │ ││ resource (Brick)                                  │
│ ╰────────────────────────────────────────────────────────╯ ││    - Consistent with my strategy of optimizing    │
│ ╭─────────────────── inventory_value ────────────────────╮ ││ resource value                                    │
│ │                                                        │ ││                                                   │
│ │          ╷        ╷         ╷                          │ ││ Decision:                                         │
│ │   Player │ change │ current │ initial                  │ ││ [Accept]                                          │
│ │  ════════╪════════╪═════════╪═════════                 │ ││                                                   │
│ │   sonnet │ 7      │ 1504    │ 1497                     │ ││ Rationale:                                        │
│ │   haiku  │ 20     │ 1064    │ 1044                     │ ││ - The trade is nearly value-neutral               │
│ │          ╵        ╵         ╵                          │ ││ - I'm converting low-value resources into a       │
│ │                                                        │ ││ higher-value resource                             │
│ ╰────────────────────────────────────────────────────────╯ ││ - I have ample surplus of Wood and Wheat          │
│                                                            ││ - This continues to improve my resource portfolio │
│                                                       

step complete
Available tools: [{'input_schema': {'type': 'object', 'properties': {'code': {'type': 'string'}}, 'required': ['code'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, 'name': 'run_code', 'description': 'Run python code in a secure sandbox by E2B. Using the Jupyter Notebook syntax.'}]
